In [1]:
import numpy as np
import tensorflow as tf
import keras
import os

2022-02-15 17:08:14.401768: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
class DataGenerator(tf.compat.v2.keras.utils.Sequence):
    def __init__(self, data_path, ann_path, list_files, list_ann_files, 
                 batch_size=64, dim=(3000,5,2), n_classes=6, shuffle=True):
        # Constructor of the data generator.
        self.dim = dim
        self.batch_size = batch_size
        self.data_path = data_path
        self.ann_path = ann_path
        self.list_files = list_files
        self.list_ann_files = list_ann_files
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.get_cnts() #Get the data count for each file        
        self.on_epoch_end() #Initialize file indexes        
        

    def __len__(self):
        # Denotes the number of batches per epoch
        return int((self.total_len+1) / self.batch_size)
    

    def __getitem__(self, index):
        
        start = index*self.batch_size
        end = min((index+1)*self.batch_size, self.total_len)
        
        X = np.empty((end - start,) + self.dim, dtype=np.float32)
        y = np.empty((end - start,), dtype=np.int32)
        
        curr_file_idx, accum_start, accum_end = self.get_accum_idx(index)
        
        curr_file = self.list_files[self.file_indexes[curr_file_idx]]
        curr_ann_file = self.list_ann_files[self.file_indexes[curr_file_idx]]
        curr_np = np.load(os.path.join(self.data_path, curr_file))
        curr_ann = np.load(os.path.join(self.ann_path, curr_ann_file))        
        
        X_1 = curr_np[start - accum_start:end - accum_start] 
        y_1 = curr_ann[start - accum_start:end - accum_start]
        from_curr = min(accum_end - start, end - start)
        X[:from_curr] = X_1
        y[:from_curr] = y_1
        
        if end > accum_end:
            curr_file_idx += 1
            accum_start = accum_end
            accum_end += self.list_cnt[self.file_indexes[curr_file_idx]]
            curr_file = self.list_files[self.file_indexes[curr_file_idx]]
            curr_ann_file = self.list_ann_files[self.file_indexes[curr_file_idx]]
            curr_np = np.load(os.path.join(self.data_path, curr_file))
            curr_ann = np.load(os.path.join(self.ann_path, curr_ann_file))
            
            X_2 = curr_np[:end - accum_start]
            y_2 = curr_ann[:end - accum_start]
            X[from_curr:] = X_2
            y[from_curr:] = y_2
        #X = np.expand_dims(X, axis=-1)
        return X, y
    '''
    # Get the next data and annotations
    # Update accumulated data count
    def update_pointers(self):
        self.curr_file_idx += 1
        curr_file = self.list_files[self.file_indexes[self.curr_file_idx]]
        curr_ann_file = self.list_ann_files[self.file_indexes[self.curr_file_idx]]
        self.curr_np = np.load(os.path.join(self.data_path, curr_file))
        self.curr_ann = np.load(os.path.join(self.ann_path, curr_ann_file))
        self.accum_start = self.accum_end
        self.accum_end += self.list_cnt[self.file_indexes[self.curr_file_idx]]   
    ''' 
    def get_accum_idx(self, index):
        curr_file_idx = 0
        accum_start = 0
        accum_end = self.list_cnt[self.file_indexes[0]]
        for i in range(len(self.file_indexes)):
            if index * self.batch_size < accum_end:
                curr_file_idx = i                
                break            
            accum_start += self.list_cnt[self.file_indexes[i]]
            accum_end += self.list_cnt[self.file_indexes[i+1]]
        
        return curr_file_idx, accum_start, accum_end
        

    def on_epoch_end(self):        
        self.curr_file_idx = 0
        # This function is called at the end of each epoch.
        self.file_indexes = np.arange(len(self.list_files)) #This is necessary to shuffle files
        if self.shuffle == True:
            np.random.shuffle(self.file_indexes)
            
        self.accum_start = 0 
        self.accum_end = self.list_cnt[self.file_indexes[0]]         
        self.curr_np = np.load(os.path.join(self.data_path, self.list_files[self.file_indexes[0]]))
        self.curr_ann = np.load(os.path.join(self.ann_path, self.list_ann_files[self.file_indexes[0]]))
            
    def get_cnts(self):
        list_cnt = []
        for f in self.list_files:
            temp_np = np.load(os.path.join(self.data_path, f))
            cnt_data = temp_np.shape[0]
            list_cnt.append(cnt_data)
            
        self.list_cnt = list_cnt
        self.total_len = sum(list_cnt)    

In [3]:
curr_path = os.getcwd() + '/'
PROCESSED_DATA_PATH = os.path.join(curr_path,'origin_npy')
HT1D_ST_path = os.path.join(PROCESSED_DATA_PATH, 'HT1D_ST')
save_signals_path_ST = os.path.join(PROCESSED_DATA_PATH,'signals_ST')
save_annotations_path_ST = os.path.join(PROCESSED_DATA_PATH,'annotations_ST')

In [4]:
def match_annotations_npy(dirname, filename):
    search_filename = filename.split('-')[0][:-2]
    file_list = os.listdir(dirname)
    filenames = [file for file in file_list if search_filename in file if file.endswith('.npy')]

    return filenames

In [5]:
dim_HT1D = (3000,5,2)
n_classes=6
epochs = 100
bs = 64
list_files = [f for f in os.listdir(HT1D_ST_path) if f.endswith('.npy')]
list_ann_files = []
for f in list_files:
    ann_file = match_annotations_npy(save_annotations_path_ST, f)
    list_ann_files.append(ann_file[0])

In [6]:
split_cnt = int(len(list_files) * 0.8)
list_files_train = list_files[:split_cnt]
list_files_test = list_files[split_cnt:]
list_ann_files_train = list_ann_files[:split_cnt]
list_ann_files_test = list_ann_files[split_cnt:]

#list_files_train = list_files[1:7,9:30]
#list_files_test = list_files[30:]
#list_ann_files_train = list_ann_files[1:7,9:30]
#list_ann_files_test = list_ann_files[30:]

In [7]:
train_generator = DataGenerator(HT1D_ST_path, save_annotations_path_ST, list_files_train, list_ann_files_train, 
                          batch_size=bs, dim=dim_HT1D, n_classes=n_classes, shuffle=False)

In [8]:
test_generator = DataGenerator(HT1D_ST_path, save_annotations_path_ST, list_files_test, list_ann_files_test, 
                          batch_size=bs, dim=dim_HT1D, n_classes=n_classes, shuffle=False)

In [9]:
list_ann_files_train

['ST7022JM-Hypnogram.npy',
 'ST7192JR-Hypnogram.npy',
 'ST7162JM-Hypnogram.npy',
 'ST7222JA-Hypnogram.npy',
 'ST7182JR-Hypnogram.npy',
 'ST7101JE-Hypnogram.npy',
 'ST7062JR-Hypnogram.npy',
 'ST7181JR-Hypnogram.npy',
 'ST7152JA-Hypnogram.npy',
 'ST7051JA-Hypnogram.npy',
 'ST7211JJ-Hypnogram.npy',
 'ST7012JP-Hypnogram.npy',
 'ST7111JE-Hypnogram.npy',
 'ST7061JR-Hypnogram.npy',
 'ST7102JE-Hypnogram.npy',
 'ST7072JA-Hypnogram.npy',
 'ST7201JO-Hypnogram.npy',
 'ST7151JA-Hypnogram.npy',
 'ST7221JA-Hypnogram.npy',
 'ST7202JO-Hypnogram.npy',
 'ST7191JR-Hypnogram.npy',
 'ST7011JP-Hypnogram.npy',
 'ST7052JA-Hypnogram.npy',
 'ST7171JA-Hypnogram.npy',
 'ST7082JW-Hypnogram.npy',
 'ST7241JO-Hypnogram.npy',
 'ST7081JW-Hypnogram.npy',
 'ST7172JA-Hypnogram.npy',
 'ST7091JE-Hypnogram.npy']

In [10]:
list_ann_files_train

['ST7022JM-Hypnogram.npy',
 'ST7192JR-Hypnogram.npy',
 'ST7162JM-Hypnogram.npy',
 'ST7222JA-Hypnogram.npy',
 'ST7182JR-Hypnogram.npy',
 'ST7101JE-Hypnogram.npy',
 'ST7062JR-Hypnogram.npy',
 'ST7181JR-Hypnogram.npy',
 'ST7152JA-Hypnogram.npy',
 'ST7051JA-Hypnogram.npy',
 'ST7211JJ-Hypnogram.npy',
 'ST7012JP-Hypnogram.npy',
 'ST7111JE-Hypnogram.npy',
 'ST7061JR-Hypnogram.npy',
 'ST7102JE-Hypnogram.npy',
 'ST7072JA-Hypnogram.npy',
 'ST7201JO-Hypnogram.npy',
 'ST7151JA-Hypnogram.npy',
 'ST7221JA-Hypnogram.npy',
 'ST7202JO-Hypnogram.npy',
 'ST7191JR-Hypnogram.npy',
 'ST7011JP-Hypnogram.npy',
 'ST7052JA-Hypnogram.npy',
 'ST7171JA-Hypnogram.npy',
 'ST7082JW-Hypnogram.npy',
 'ST7241JO-Hypnogram.npy',
 'ST7081JW-Hypnogram.npy',
 'ST7172JA-Hypnogram.npy',
 'ST7091JE-Hypnogram.npy']

In [11]:
x = np.load(os.path.join(HT1D_ST_path, list_files_train[0]))
y = np.load(os.path.join(save_annotations_path_ST, list_ann_files_train[0]))
x.shape[0] % 64, y.shape[0] % 64

(48, 48)

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=dim_HT1D),
    keras.layers.Conv2D(filters=64, kernel_size=(3,5), padding="valid"),
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Reshape((2998, 64)),
    keras.layers.Conv1D(filters=64, kernel_size=100, padding="valid"),
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.ReLU(),
    keras.layers.Conv1D(filters=128, kernel_size=100, padding="valid"),
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.ReLU(),
    keras.layers.Conv1D(filters=128, kernel_size=100, padding="valid"),
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(n_classes, activation="softmax")
])

2022-02-15 17:09:20.973599: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-02-15 17:09:21.100768: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2022-02-15 17:09:21.111351: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-02-15 17:09:22.464270: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-02-15 17:09:22.464395: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2022-02-15 17:09:22.752239: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcu

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper (ModuleWrappe (None, 2998, 1, 64)       1984      
_________________________________________________________________
batch_normalization (BatchNo (None, 2998, 1, 64)       256       
_________________________________________________________________
re_lu (ReLU)                 (None, 2998, 1, 64)       0         
_________________________________________________________________
reshape (Reshape)            (None, 2998, 64)          0         
_________________________________________________________________
module_wrapper_1 (ModuleWrap (None, 2899, 64)          409664    
_________________________________________________________________
batch_normalization_1 (Batch (None, 2899, 64)          256       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 2899, 64)          0

In [13]:
callbacks = []

#Checkpoint설정
checkpoint_dir = './ckpt_conv1D'
model_cp_path = os.path.join(checkpoint_dir, "ckpt_{epoch}")
callbacks.append(tf.keras.callbacks.ModelCheckpoint(model_cp_path, save_weights_only=True))


#Learning rate 스케쥴 설정
#callbacks.append(LearningRateScheduler(decay))

#General logs on csv
#callbacks.append(CSVLogger(model_csv_path)) 

In [20]:
def eval_model(test_gen, model):
    total_cnt = 0.0
    total_loss = 0.0
    correct = 0.0
    for x, y in test_gen:
        y_pred = model.predict(x)
        y_pred_cls = tf.math.argmax(y_pred, axis=-1)
        correct += tf.reduce_sum(tf.cast(tf.equal(y_pred_cls, y), tf.float32))
        total_cnt += y_pred.shape[0]
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, 
                                                                reduction=tf.keras.losses.Reduction.SUM)
        y = tf.cast(y, dtype=tf.int32)
        total_loss += loss_fn(y, y_pred).numpy()
            
    test_acc = correct / total_cnt
    test_loss = total_loss / total_cnt
    
    print("test_acc: %.3f, test_loss: %.6f"%(test_acc, test_loss))

In [21]:
class eval_callback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        eval_model(test_generator, model)

In [22]:
callbacks.append(eval_callback())

In [23]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=['sparse_categorical_accuracy'],
)

In [24]:
history = model.fit(train_generator,              
              epochs=epochs,
              verbose=1,
              callbacks=callbacks)

2022-02-14 20:02:25.472634: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-14 20:02:25.493364: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2500295000 Hz


Epoch 1/100


2022-02-14 20:02:26.417965: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2022-02-14 20:02:26.961612: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8201
2022-02-14 20:02:27.846387: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-02-14 20:02:27.846460: W tensorflow/stream_executor/gpu/asm_compiler.cc:56] Couldn't invoke ptxas --version
2022-02-14 20:02:27.846877: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-02-14 20:02:27.846963: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-02-14 20:02:27.881628: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully ope

445/445 [==============================] - 38s 73ms/step - loss: 13.8920 - sparse_categorical_accuracy: 0.3215
test_acc: 0.479, test_loss: 1.761373
Epoch 2/100
445/445 [==============================] - 33s 74ms/step - loss: 1.9383 - sparse_categorical_accuracy: 0.4206
test_acc: 0.479, test_loss: 1.705752
Epoch 3/100
445/445 [==============================] - 33s 73ms/step - loss: 1.7035 - sparse_categorical_accuracy: 0.4533
test_acc: 0.479, test_loss: 1.571115
Epoch 4/100
445/445 [==============================] - 33s 74ms/step - loss: 1.7792 - sparse_categorical_accuracy: 0.4654
test_acc: 0.479, test_loss: 1.983801
Epoch 5/100
445/445 [==============================] - 33s 73ms/step - loss: 1.5471 - sparse_categorical_accuracy: 0.4644
test_acc: 0.479, test_loss: 2.376467
Epoch 6/100
445/445 [==============================] - 33s 73ms/step - loss: 1.3702 - sparse_categorical_accuracy: 0.4828
test_acc: 0.479, test_loss: 1.683532
Epoch 7/100
445/445 [==============================] - 33

KeyboardInterrupt: 